# Import

In [691]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from matplotlib import rc
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import matplotlib.pyplot as plt
import re
import glob
import os
from pathlib import Path

# Read csv files

In [692]:
submission = pd.read_csv('../../data/sample_submission.csv')
holidays = pd.read_csv('../../data/holidays_2023_2025.csv')
train = pd.read_csv('../../data/train/train.csv')
# os.getcwd()
terms = pd.read_csv('../solar_term_2023_2025.csv')

# Feature about date
1. year
2. month
3. day
4. weekday
5. is_holiday
6. is_sandwich
7. is_before_holiday
8. is_after_holiday
9. is_weekend

In [693]:
# 1) 날짜 범위
dates = pd.date_range(start="2023-01-01", end="2025-05-31", freq="D")

# 연, 월, 일 col 생성
train['영업일자'] = pd.to_datetime(train['영업일자'])
train['time_idx'] = (train['영업일자'] - train['영업일자'].min()).dt.days
train['year'] = train['영업일자'].dt.year
train['month'] = train['영업일자'].dt.month
train['day'] = train['영업일자'].dt.day
# 영업장, 메뉴명 col 생성
train[['영업장명', '메뉴명']] = train['영업장명_메뉴명'].str.split('_', expand=True)
# 요일 col 생성
train['weekday'] = train['영업일자'].dt.weekday.astype(int)

# 공휴일 col 생성
holidays["locdate"] = pd.to_datetime(holidays["locdate"])
holiday_dates = set(holidays['locdate'])
train['is_holiday'] = train['영업일자'].isin(holiday_dates).astype(int)

# 5) isSandwich: 오늘은 평일(0)이고, 어제/내일이 모두 쉬는 날(1)인 경우 1
train["is_sandwich"] = 0
train.loc[
    (train["is_holiday"] == 0) &
    (train["is_holiday"].shift(1) == 1) &
    (train["is_holiday"].shift(-1) == 1),
    "is_sandwich"
] = 1

In [694]:
train['is_before_holiday'] = train['is_holiday'].shift(-1).fillna(0)
train['is_before_holiday'] = train['is_before_holiday'].astype(int)
train['is_after_holiday'] = train['is_holiday'].shift(1).fillna(0)
train['is_after_holiday'] = train['is_after_holiday'].astype(int)
train['is_weekend'] = train['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)

# Feature about top menu --todo--

In [695]:
# 예시: train 데이터에 영업장명, 메뉴명, 매출수량 컬럼이 있다고 가정
# train = pd.read_csv("train.csv")

# 1) 영업장별 메뉴별 총 매출수량 계산
menu_sales = (
    train.groupby(['영업장명', '메뉴명'], as_index=False)['매출수량']
         .sum()
)

# 2) 영업장별 매출수량 상위 3개 메뉴 추출
top3_menus = (
    menu_sales.groupby('영업장명')
              .apply(lambda g: g.nlargest(3, '매출수량'))
              .reset_index(drop=True)
)

# 3) Top3 여부를 dict 형태로 만들어 매핑
top3_set = set(zip(top3_menus['영업장명'], top3_menus['메뉴명']))

# 4) train에 is_popular 플래그 추가
train['is_popular'] = train.apply(
    lambda row: 1 if (row['영업장명'], row['메뉴명']) in top3_set else 0,
    axis=1
)

/var/folders/3t/x_vcdczj5fn_v1nnkppzjpwh0000gn/T/ipykernel_2586/2046610547.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.nlargest(3, '매출수량'))


# menu_ category

In [696]:
menu_category = {
    '1인 수저세트': '기타',
    'BBQ55(단체)': '메인메뉴',
    '대여료 60,000원': '기타',
    '대여료 30,000원': '기타',
       '대여료 90,000원': '기타',
       '본삼겹 (단품,실내)': '메인메뉴',
       '스프라이트 (단체)': '음료',
       '신라면': '추가메뉴',
       '쌈야채세트': '추가메뉴',
       '쌈장': '추가메뉴',
       '육개장 사발면': '추가메뉴',
       '일회용 소주컵': '기타',
       '일회용 종이컵': '기타',
       '잔디그늘집 대여료 (12인석)': '기타',
       '잔디그늘집 대여료 (6인석)': '기타',
       '잔디그늘집 의자 추가': '기타',
       '참이슬 (단체)': '주류',
       '친환경 접시 14cm': '기타',
       '친환경 접시 23cm': '기타',
       '카스 병(단체)': '주류',
       '콜라 (단체)': '음료',
       '햇반': '추가메뉴',
       '허브솔트': '추가메뉴',
       '(단체) 공깃밥': '추가메뉴',
       '(단체) 생목살 김치전골 2.0': '메인메뉴',
       '(단체) 은이버섯 갈비탕': '메인메뉴',
       '(단체) 한우 우거지 국밥': '메인메뉴',
       '(단체) 황태해장국 3/27까지': '메인메뉴',
       '(정식) 된장찌개': '메인메뉴',
       '(정식) 물냉면 ': '메인메뉴',
       '(정식) 비빔냉면': '메인메뉴',
       '(후식) 된장찌개': '추가메뉴',
       '(후식) 물냉면': '추가메뉴',
       '(후식) 비빔냉면': '추가메뉴',
       '갑오징어 비빔밥': '메인메뉴',
       '갱시기': '메인메뉴',
       '공깃밥': '추가메뉴',
       '꼬막 비빔밥': '메인메뉴',
       '느린마을 막걸리': '주류',
       '담하 한우 불고기': '메인메뉴',
       '담하 한우 불고기 정식': '메인메뉴',
       '더덕 한우 지짐': '메인메뉴',
       '들깨 양지탕': '메인메뉴',
       '라면사리': '추가메뉴',
       '룸 이용료': '기타',
       '메밀면 사리': '추가메뉴',
       '명인안동소주': '주류',
       '명태회 비빔냉면': '메인메뉴',
       '문막 복분자 칵테일': '주류',
       '봉평메밀 물냉면': '메인메뉴',
       '생목살 김치찌개': '메인메뉴',
       '스프라이트': '음료',
       '은이버섯 갈비탕': '메인메뉴',
       '제로콜라': '음료',
       '참이슬': '주류',
       '처음처럼': '주류',
       '카스': '주류',
       '콜라': '음료',
       '테라': '주류',
       '하동 매실 칵테일': '주류',
       '한우 떡갈비 정식': '메인메뉴',
       '한우 미역국 정식': '메인메뉴',
       '한우 우거지 국밥': '메인메뉴',
       '한우 차돌박이 된장찌개': '메인메뉴',
       '황태해장국': '메인메뉴',
       'AUS (200g)': '메인메뉴',
       'G-Charge(3)': '기타',
       'Gls.Sileni': '주류',
       'Gls.미션 서드': '주류',
       'Open Food': '기타',
       '그릴드 비프 샐러드': '메인메뉴',
       '까르보나라': '메인메뉴',
       '모둠 해산물 플래터': '메인메뉴',
       '미션 서드 카베르네 쉬라': '메인메뉴',
       '버섯 크림 리조또': '메인메뉴',
       '빵 추가 (1인)': '추가메뉴',
       '시저 샐러드 ': '메인메뉴',
       '아메리카노': '음료',
       '알리오 에 올리오 ': '메인메뉴',
       '양갈비 (4ps)': '메인메뉴',
       '자몽리치에이드': '음료',
       '하이네켄(생)': '주류',
       '한우 (200g)': '메인메뉴',
       '해산물 토마토 리조또': '메인메뉴',
       '해산물 토마토 스튜 파스타': '메인메뉴',
       '해산물 토마토 스파게티': '메인메뉴',
       '(단체)브런치주중 36,000': '메인메뉴',
       '(오븐) 하와이안 쉬림프 피자': '메인메뉴',
       '(화덕) 불고기 페퍼로니 반반피자': '메인메뉴',
       'BBQ Platter': '메인메뉴',
       'BBQ 고기추가': '추가메뉴',
       '글라스와인 (레드)': '주류',
       '레인보우칵테일(알코올)': '주류',
       '미라시아 브런치 (패키지)': '메인메뉴',
       '버드와이저(무제한)': '주류',
       '보일링 랍스타 플래터': '메인메뉴',
       '보일링 랍스타 플래터(덜매운맛)': '메인메뉴',
       '브런치 2인 패키지 ': '메인메뉴',
       '브런치 4인 패키지 ': '메인메뉴',
       '브런치(대인) 주말': '메인메뉴',
       '브런치(대인) 주중': '메인메뉴',
       '브런치(어린이)': '메인메뉴',
       '쉬림프 투움바 파스타': '메인메뉴',
       '스텔라(무제한)': '주류',
       '애플망고 에이드': '음료',
       '얼그레이 하이볼': '주류',
       '오븐구이 윙과 킬바사소세지': '메인메뉴',
       '유자 하이볼': '주류',
       '잭 애플 토닉': '주류',
       '칠리 치즈 프라이': '추가메뉴',
       '코카콜라': '음료',
       '코카콜라(제로)': '음료',
       '콥 샐러드': '추가메뉴',
       '파스타면 추가(150g)': '추가메뉴',
       '핑크레몬에이드': '음료',
       'Cass Beer': '주류',
       'Conference L1': '연회장 대여',
       'Conference L2': '연회장 대여',
       'Conference L3': '연회장 대여',
       'Conference M1': '연회장 대여',
       'Conference M8': '연회장 대여',
       'Conference M9': '연회장 대여',
       'Convention Hall': '연회장 대여',
       'Cookie Platter': '디저트',
       'Grand Ballroom': '연회장 대여',
       'OPUS 2': '연회장 대여',
       'Regular Coffee': '음료',
       '골뱅이무침': '메인메뉴',
       '돈목살 김치찌개 (밥포함)': '메인메뉴',
       '로제 치즈떡볶이': '메인메뉴',
       '마라샹궈': '메인메뉴',
       '매콤 무뼈닭발&계란찜': '메인메뉴',
       '모둠 돈육구이(3인)': '메인메뉴',
       '삼겹살추가 (200g)': '추가메뉴',
       '야채추가': '추가메뉴',
       '왕갈비치킨': '메인메뉴',
       '주먹밥 (2ea)': '추가메뉴',
       '공깃밥(추가)': '추가메뉴',
       '구슬아이스크림': '디저트',
       '단체식 13000(신)': '메인메뉴',
       '단체식 18000(신)': '메인메뉴',
       '돼지고기 김치찌개': '메인메뉴',
       '복숭아 아이스티': '음료',
       '새우 볶음밥': '메인메뉴',
       '새우튀김 우동': '메인메뉴',
       '샷 추가': '추가메뉴',
       '수제 등심 돈까스': '메인메뉴',
       '아메리카노(HOT)': '음료',
       '아메리카노(ICE)': '음료',
       '약 고추장 돌솥비빔밥': '메인메뉴',
       '어린이 돈까스': '메인메뉴',
       '오픈푸드': '기타',
       '진사골 설렁탕': '메인메뉴',
       '짜장면': '메인메뉴',
       '짜장밥': '메인메뉴',
       '짬뽕': '메인메뉴',
       '짬뽕밥': '메인메뉴',
       '치즈돈까스': '메인메뉴',
       '카페라떼(HOT)': '음료',
       '카페라떼(ICE)': '음료',
       '한상 삼겹구이 정식(2인) 소요시간 약 15~20분': '메인메뉴',
       '꼬치어묵': '메인메뉴',
       '떡볶이': '메인메뉴',
       '생수': '음료',
       '치즈 핫도그': '디저트',
       '페스츄리 소시지': '디저트',
       '단호박 식혜 ': '음료',
       '병천순대': '메인메뉴',
       '참살이 막걸리': '주류',
       '찹쌀식혜': '음료',
       '해물파전': '메인메뉴',
       '메밀미숫가루': '음료',
       '아메리카노 HOT': '음료',
       '아메리카노 ICE': '음료',
       '카페라떼 ICE': '음료',
       '현미뻥스크림': '디저트'

}


In [697]:
# Assign the menu_category to df
train['menu_category'] = train['메뉴명'].map(menu_category)
train['menu_category'], uniques = pd.factorize(train['menu_category'])

# avg_sales_all_days

In [698]:
menu_avg = train.groupby('메뉴명')['매출수량'].mean()
train['avg_sales_all_days'] = train['메뉴명'].map(menu_avg)

# Feature about zero
1. avg_sales_nonzero_days
2. zero_sales_day_ratio
3. is_sparse_menu

In [699]:
nonzero_avg = train[train['매출수량'] > 0].groupby('메뉴명')['매출수량'].mean()
# Assign the nonzero_avg to df
train['avg_sales_nonzero_days'] = train['메뉴명'].map(nonzero_avg)

In [700]:
zero_ratio = train.groupby('메뉴명')['매출수량'].apply(lambda x: (x.eq(0).sum() / len(x)) * 100)

# Assign the zero_ratio to df
train['zero_sales_day_ratio'] = train['메뉴명'].map(zero_ratio)

In [701]:
train['is_sparse_menu'] = np.where(train['zero_sales_day_ratio'] > 50, 1, 0)

# Feature about 연회장
1. banquet_type

In [702]:
df_연회장 = train[train['영업장명']=='연회장'].pivot_table(index='영업일자',columns='메뉴명',values='매출수량', aggfunc = 'sum').reset_index()

In [703]:
df_연회장['연회장 대여'] = df_연회장[['Conference L1','Conference L2','Conference L3','Conference M1','Conference M8','Conference M9','Convention Hall','Grand Ballroom','OPUS 2']].sum(axis=1)
df_연회장['음료 및 쿠키'] = df_연회장[['Cookie Platter','Cass Beer','Regular Coffee']].sum(axis=1)
df_연회장['음식'] = df_연회장[['골뱅이무침','공깃밥','돈목살 김치찌개 (밥포함)','로제 치즈떡볶이','마라샹궈','매콤 무뼈닭발&계란찜','모둠 돈육구이(3인)','삼겹살추가 (200g)','야채추가','왕갈비치킨','주먹밥 (2ea)']].sum(axis=1)

In [704]:
def banquet_type(row):
    if row['연회장 대여'] > 0 and row['음식'] == 0 and row['음료 및 쿠키'] == 0:
        return 1 # 대여만
    elif row['연회장 대여'] == 0 and row['음식'] == 0 and row['음료 및 쿠키'] > 0:
        return 2 # 음료및쿠키만
    elif row['연회장 대여'] == 0 and row['음식'] > 0 and row['음료 및 쿠키'] == 0:
        return 3 # 음식만
    elif row['연회장 대여'] > 0 and row['음식'] == 0 and row['음료 및 쿠키'] > 0:
        return 4 # 대여+음료및쿠키
    elif row['연회장 대여'] > 0 and row['음식'] > 0 and row['음료 및 쿠키'] == 0:
        return 5 # 대여+음식
    elif row['연회장 대여'] == 0 and row['음식'] > 0 and row['음료 및 쿠키'] > 0:
        return 6 # 음식+음료및쿠키
    elif row['연회장 대여'] > 0 and row['음식'] > 0 and row['음료 및 쿠키'] > 0:
        return 7 # 대여+음료및쿠키+음식
    else:
        return 0 # 연회장 총매출이 0인경우
df_연회장['banquet_type'] = df_연회장.apply(banquet_type, axis=1)


# Features
1. is_drink
2. is_alcohol
3. is_set_menu

In [705]:
drink_keywords = ['콜라', '스프라이트', '제로콜라', '자몽리치에이드', '애플망고 에이드', '핑크레몬에이드', '아메리카노',
                  '식혜', '메밀미숫가루', '아메리카노', '카페라떼', '복숭아 아이스티','샷 추가',
                  '생수']

alcohol_keywords = ['Gls.Sileni', 'Gls.미션 서드', '미션 서드 카메르네 쉬라', '하이네켄', '막걸리',
                    '와인', '버드와이저', '스텔라', '하이볼', '잭 애플 토닉', '참이슬', '소주', '처음처럼',
                    '카스', '테라', '칵테일', 'Cass']

set_keywords = ['정식']

train['is_drink'] = train['영업장명_메뉴명'].apply(
    lambda x: 1 if any(keyword in str(x) for keyword in drink_keywords) else 0
)

train['is_alcohol'] = train['영업장명_메뉴명'].apply(
    lambda x: 1 if (
        any(keyword in str(x) for keyword in alcohol_keywords)
        and '컵' not in str(x)
    ) else 0
)

train['is_set_menu'] = train['영업장명_메뉴명'].apply(
    lambda x: 1 if (
        any(keyword in str(x) for keyword in set_keywords)
    ) else 0
)

#매출수량이 문자열이면 숫자로 변환
train['매출수량'] = pd.to_numeric(train['매출수량'], errors='coerce')


# demand_volatility, demand_stability

In [706]:
menu_stats=(
    train.groupby('영업장명_메뉴명')['매출수량']
    .agg(['mean','std'])
    .reset_index()
)

menu_stats['demand_volatility']=menu_stats['std']/menu_stats['mean']
menu_stats['demand_stability']=1/menu_stats['demand_volatility']

menu_stats.rename(columns={'mean':'평균매출수량','std':'표준편차'},inplace=True)

# Merge menu_stats back to df
train=train.merge(menu_stats[['영업장명_메뉴명', 'demand_volatility','demand_stability']], on='영업장명_메뉴명',how='left')

# Add New

In [707]:
train['is_nonzero'] = (train['매출수량'] > 0).astype(int)

# **데이터 전처리**

1. **is_spike**: 전식당, 변할 수 있는 가능성을 학습하게끔 하는 것, 당일 수치가 최근 7일간 평균 + 2 × 표준편차보다 크면 1, 아니면 0, 위로 갑자기 튀는 것을 포착
2. **is_drop**: 전식당, 변할 수 있는 가능성을 학습하게끔 하는 것, 당일 수치가 최근 7일간 평균 − 2 × 표준편차보다 작으면 1, 아니면 0, 아래로 갑자기 튀는 것을 포착
3. **is_weekday_price**: 미라시아, 요금제가 주중 기준인지 여부 구분	메뉴명에 '주중'이 포함되면 1, 아니면 0, 주중 요금이 적용된 메뉴인지 여부
4. **is_weekend_price**: 미라시아, 요금제가 주말 기준인지 여부 구분	메뉴명에 '주말'이 포함되면 1, 아니면 0, 주말 요금이 적용된 메뉴인지 여부
5. **seasonal_index**: 전식당,

    • 1분기 (Q1): 1월 1일부터 3월 31일까지

    • 2분기 (Q2): 4월 1일부터 6월 30일까지

    • 3분기 (Q3): 7월 1일부터 9월 30일까지

    • 4분기 (Q4): 10월 1일부터 12월 31일까지

    월별 또는 분기별 매출 패턴 분석하여 생성, 분기별 매출 수치화
6. **미라시아 단체 관련 변수( brunch_flag, hallroom_flag)** :
    
    6-1. **brunch_flag**: 단체 브런치 메뉴 매출이 생긴 날의 플래그, 연회장_룸타입에만 플래그를 세운다.

    6-2. **hallroom_flag**: 연회장_룸타입 매출이 생긴 날의 플래그, (단체)브런치주중 36,000 에만 플래그를 세운다.

# quarter_index(all)

생성 목적: 분기별 매출 수치화

생성 방법:

• 1분기 (Q1): 1월 1일부터 3월 31일까지

• 2분기 (Q2): 4월 1일부터 6월 30일까지

• 3분기 (Q3): 7월 1일부터 9월 30일까지

• 4분기 (Q4): 10월 1일부터 12월 31일까지

분기별 매출 패턴 분석하여 누적합으로 생성

In [708]:
# # 0) 날짜 보정
# if df['영업일자'].dtype == 'O':
#     df['영업일자'] = pd.to_datetime(df['영업일자'], errors='coerce')

# 1) '분기' → 'quarter'로 변경 (없으면 새로 생성)
#     Q1, Q2, Q3, Q4 형식으로 생성
train['quarter'] = train['영업일자'].dt.to_period('Q').astype(str).str[-2:]

In [709]:
def add_time_feats(df: pd.DataFrame, date_col: str = '영업일자') -> pd.DataFrame:
    # 사본에서 계산 (원본 보존)
    df = df.copy()
    # 날짜 캐스팅
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')

    # 기본 파생
    year  = df[date_col].dt.year.astype('Int16')
    month = df[date_col].dt.month.astype('Int8')
    day   = df[date_col].dt.day.astype('Int8')

    # 분기/분기 내 진행일
    q = df[date_col].dt.to_period('Q')
    q_start = q.dt.start_time
    quarter = q.astype(str).str[-2:].map({'Q1':0,'Q2':1,'Q3':2,'Q4':3}).astype('Int8')
    day_of_quarter = (df[date_col] - q_start).dt.days.astype('Int16')

    # 결과 합치기
    df['year'] = year
    df['month'] = month
    df['day'] = day
    df['quarter'] = quarter
    df['day_of_quarter'] = day_of_quarter
    return df

In [710]:
# 0) 시간 피처
train = add_time_feats(train, '영업일자')  # 이미 정의된 함수 사용
train = train.sort_values(['영업장명_메뉴명', '영업일자']).copy()

# 1) 분기 내 누적 비율
grp = ['영업장명_메뉴명', 'year', 'quarter']
train['quarter_cum'] = train.groupby(grp)['매출수량'].cumsum()
train['quarter_tot'] = train.groupby(grp)['매출수량'].transform('sum')
train['cum_share_actual'] = (train['quarter_cum'] / train['quarter_tot']).astype('float32')

# 2) 참조 곡선: (매장메뉴, day_of_quarter) 중앙값
curve_ref = (
    train.groupby(['영업장명_메뉴명','day_of_quarter'])['cum_share_actual']
         .median()
         .reset_index()
         .rename(columns={'cum_share_actual':'cum_share_ref_doq'})
)
curve_ref['day_of_quarter'] = curve_ref['day_of_quarter'].astype('Int16')

# 2-보강) 전역 백업값
global_md_doq = (
    train.groupby(['day_of_quarter'])['cum_share_actual']
         .median()
         .reset_index()
         .rename(columns={'cum_share_actual':'cum_share_doq_md'})
)
global_md_doq['day_of_quarter'] = global_md_doq['day_of_quarter'].astype('Int16')

global_md_month_day = (
    train.groupby(['month','day'])['cum_share_actual']
         .median()
         .reset_index()
         .rename(columns={'cum_share_actual':'cum_share_global_md'})
)
global_md_month_day['month'] = global_md_month_day['month'].astype('Int8')
global_md_month_day['day']   = global_md_month_day['day'].astype('Int8')

# 키 dtype 정리
train['month']          = train['month'].astype('Int8')
train['day']            = train['day'].astype('Int8')
train['day_of_quarter'] = train['day_of_quarter'].astype('Int16')

# 기존 cum_share_ref 제거
train = train.drop(columns=['cum_share_ref'], errors='ignore')

# 3) 매핑: 우선 doq → 전역 doq → (month,day) → 0.5
train = train.merge(
    curve_ref, on=['영업장명_메뉴명','day_of_quarter'],
    how='left', validate='many_to_one'
)
train = train.merge(
    global_md_doq, on='day_of_quarter',
    how='left', validate='many_to_one'
)
train = train.merge(
    global_md_month_day, on=['month','day'],
    how='left', validate='many_to_one'
)

train['cum_share_ref'] = (
    train['cum_share_ref_doq']
        .fillna(train['cum_share_doq_md'])
        .fillna(train['cum_share_global_md'])
        .fillna(0.5)
).astype('float32')

# 4) (강력 추천) 분기 내 단조(비감소) 보정
train = train.sort_values(['영업장명_메뉴명','year','quarter','day_of_quarter'])
train['cum_share_ref'] = (
    train.groupby(['영업장명_메뉴명','year','quarter'])['cum_share_ref']
         .apply(lambda s: s.ffill().bfill().cummax().clip(0,1))
         .reset_index(level=[0,1,2], drop=True)
)

# 5) 임시 컬럼/중간 계산 정리
train = train.drop(
    columns=[
        'cum_share_ref_doq','cum_share_doq_md','cum_share_global_md',
        'quarter_cum','quarter_tot','cum_share_actual'
    ],
    errors='ignore'
)
# 필요시 day_of_quarter도 정리(남겨두고 싶으면 주석)
# train = train.drop(columns=['day_of_quarter'])

# seasonal_index

In [711]:
# month -> season (0:겨울, 1:봄, 2:여름, 3:가을)
def get_season_code(month: int) -> int:
    if month in (12, 1, 2):   return 0
    if month in (3, 4, 5):    return 1
    if month in (6, 7, 8):    return 2
    return 3  # 9~11

def add_season_feats(df: pd.DataFrame, date_col: str = '영업일자') -> pd.DataFrame:
    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col], errors='coerce')

    # 기본 날짜 파생 (NumPy 정수형으로 맞춤)
    out['year']  = out[date_col].dt.year.astype('int16')
    out['month'] = out[date_col].dt.month.astype('int8')
    out['day']   = out[date_col].dt.day.astype('int8')

    # season (0:겨울, 1:봄, 2:여름, 3:가을)
    out['season'] = out['month'].apply(get_season_code).astype('int8')

    # 시즌 시작 연도(season_year) 계산
    y = out['year'].astype('int32')
    m = out['month'].astype('int32')
    season_year = y.copy()
    season_year[(out['season'] == 0) & (m <= 2)] -= 1  # 겨울의 1~2월은 전년도
    out['season_year'] = season_year.astype('int16')

    # 시즌 시작 월(start_month) → np.select 결과는 ndarray이므로 'int8' 사용
    start_month = np.select(
        [
            out['season'].eq(0),  # 겨울
            out['season'].eq(1),  # 봄
            out['season'].eq(2),  # 여름
        ],
        [12, 3, 6],
        default=9               # 가을
    ).astype('int8')            # ★ 여기! 'Int8'이 아니라 'int8'

    # 시즌 시작일
    season_start = pd.to_datetime({
        "year":  out['season_year'].astype(int),  # to_datetime에는 일반 int가 안전
        "month": start_month.astype(int),
        "day":   1
    })

    # 시즌 내 경과일
    out['day_of_season'] = (out[date_col] - season_start).dt.days.astype('int16')

    return out

In [712]:
# 0) 시즌 피처 생성 + 정렬
train_s = add_season_feats(train, '영업일자')
train_s = train_s.sort_values(['영업장명_메뉴명', '영업일자']).copy()

# 1) 시즌 내 누적 비율 (누적/시즌합)
grp_season = ['영업장명_메뉴명', 'season_year', 'season']
train_s['season_cum'] = train_s.groupby(grp_season)['매출수량'].cumsum()
train_s['season_tot'] = train_s.groupby(grp_season)['매출수량'].transform('sum')
train_s['cum_share_actual_season'] = (train_s['season_cum'] / train_s['season_tot']).astype('float32')

# 2) 참조 곡선: (매장메뉴, day_of_season) 중앙값 (로버스트)
curve_ref_season = (
    train_s.groupby(['영업장명_메뉴명', 'day_of_season'])['cum_share_actual_season']
           .median()
           .reset_index()
           .rename(columns={'cum_share_actual_season': 'cum_share_ref_dos'})
)
curve_ref_season['day_of_season'] = curve_ref_season['day_of_season'].astype('Int16')

# 2-보강) 전역 백오프
global_md_dos = (
    train_s.groupby(['day_of_season'])['cum_share_actual_season']
           .median()
           .reset_index()
           .rename(columns={'cum_share_actual_season': 'cum_share_dos_md'})
)
global_md_dos['day_of_season'] = global_md_dos['day_of_season'].astype('Int16')

global_md_month_day = (
    train_s.groupby(['month','day'])['cum_share_actual_season']
           .median()
           .reset_index()
           .rename(columns={'cum_share_actual_season': 'cum_share_global_md'})
)
global_md_month_day['month'] = global_md_month_day['month'].astype('Int8')
global_md_month_day['day']   = global_md_month_day['day'].astype('Int8')

# 3) train에 시즌 참조 붙이기: dos → dos_global → (month,day) → 0.5
train_s = train_s.merge(
    curve_ref_season, on=['영업장명_메뉴명','day_of_season'],
    how='left', validate='many_to_one'
).merge(
    global_md_dos, on='day_of_season',
    how='left', validate='many_to_one'
).merge(
    global_md_month_day, on=['month','day'],
    how='left', validate='many_to_one'
)

train_s['cum_share_ref_season'] = (
    train_s['cum_share_ref_dos']
          .fillna(train_s['cum_share_dos_md'])
          .fillna(train_s['cum_share_global_md'])
          .fillna(0.5)
).astype('float32')

# 4) (강추) 시즌 내 단조(비감소) 보정
train_s = train_s.sort_values(['영업장명_메뉴명','season_year','season','day_of_season'])
train_s['cum_share_ref_season'] = (
    train_s.groupby(['영업장명_메뉴명','season_year','season'])['cum_share_ref_season']
           .apply(lambda s: s.ffill().bfill().cummax().clip(0,1))
           .reset_index(level=[0,1,2], drop=True)
)

# 5) 원본 train에 최종 칼럼만 붙이기(원본 순서 보존)
train = train.copy()
train['cum_share_ref_season'] = train_s.sort_index()['cum_share_ref_season'].values
train['season'] = train_s.sort_index()['season'].values

# solar_term_index

In [713]:
# --- 0) 날짜형 통일 ---
train['영업일자'] = pd.to_datetime(train['영업일자'], errors='coerce')
terms['locdate']  = pd.to_datetime(terms['locdate'],  errors='coerce')

# --- 1) 절기 구간 정의 (start/end) ---
terms = terms.sort_values('locdate').reset_index(drop=True)
terms['end_date'] = terms['locdate'].shift(-1) - pd.Timedelta(days=1)
terms.loc[terms.index[-1], 'end_date'] = train['영업일자'].max()

# --- 2) asof 머지로 절기 매핑 ---
# 왼쪽에 solar_term 있으면 충돌 방지 위해 제거
left  = train.drop(columns=['solar_term'], errors='ignore').sort_values('영업일자')
right = terms[['locdate', 'end_date', 'solar_term']].sort_values('locdate')

merged = pd.merge_asof(
    left, right,
    left_on='영업일자', right_on='locdate',
    direction='backward',
    # suffixes=('', '_r')  # (옵션) 충돌 시 오른쪽 접미사
)

# 구간 내만 유효
in_range = merged['영업일자'] <= merged['end_date']
# 이제 merged에는 'solar_term'이 반드시 존재
merged['solar_term'] = merged['solar_term'].where(in_range)

# --- 3) term_cycle_id & day_of_term 산출 ---
train_st = merged.rename(columns={'locdate':'term_start', 'end_date':'term_end'}).copy()
train_st['term_start'] = pd.to_datetime(train_st['term_start'])
train_st['term_end']   = pd.to_datetime(train_st['term_end'])
train_st['day_of_term'] = (train_st['영업일자'] - train_st['term_start']).dt.days.astype('Int16')
train_st['term_cycle_id'] = train_st['term_start']  # 각 절기 사이클 식별자

# 이하 동일 (누적/참조/백오프/단조보정/최종 부착)

# --- 4) 절기 내 누적비율(actual) 계산 ---
grp_term = ['영업장명_메뉴명', 'solar_term', 'term_cycle_id']
train_st = train_st.sort_values(['영업장명_메뉴명','영업일자'])
train_st['term_cum'] = train_st.groupby(grp_term)['매출수량'].cumsum()
train_st['term_tot'] = train_st.groupby(grp_term)['매출수량'].transform('sum')
train_st['cum_share_actual_term'] = (train_st['term_cum'] / train_st['term_tot']).astype('float32')

# --- 5) 참조곡선: (매장메뉴, solar_term, day_of_term) 중앙값 ---
curve_ref_term = (
    train_st.groupby(['영업장명_메뉴명','solar_term','day_of_term'])['cum_share_actual_term']
            .median()
            .reset_index()
            .rename(columns={'cum_share_actual_term':'cum_share_ref_dot'})  # dot = day_of_term
)
curve_ref_term['day_of_term'] = curve_ref_term['day_of_term'].astype('Int16')

# --- 6) 전역 백오프 ---
# (1) 같은 solar_term 내 day_of_term 중앙값
global_md_dot = (
    train_st.groupby(['solar_term','day_of_term'])['cum_share_actual_term']
            .median()
            .reset_index()
            .rename(columns={'cum_share_actual_term':'cum_share_term_dot_md'})
)
global_md_dot['day_of_term'] = global_md_dot['day_of_term'].astype('Int16')

# (2) 마지막 백업: (month, day) 캘린더 중앙값
train_st['month'] = train_st['영업일자'].dt.month.astype('Int8')
train_st['day']   = train_st['영업일자'].dt.day.astype('Int8')
global_md_month_day_term = (
    train_st.groupby(['month','day'])['cum_share_actual_term']
            .median()
            .reset_index()
            .rename(columns={'cum_share_actual_term':'cum_share_global_md_term'})
)

# --- 7) 참조 매핑: dot → term-dot-global → (month,day) → 0.5 ---
train_st = train_st.merge(
    curve_ref_term, on=['영업장명_메뉴명','solar_term','day_of_term'],
    how='left', validate='many_to_one'
).merge(
    global_md_dot, on=['solar_term','day_of_term'],
    how='left', validate='many_to_one'
).merge(
    global_md_month_day_term, on=['month','day'],
    how='left', validate='many_to_one'
)

train_st['cum_share_ref_solar'] = (
    train_st['cum_share_ref_dot']
           .fillna(train_st['cum_share_term_dot_md'])
           .fillna(train_st['cum_share_global_md_term'])
           .fillna(0.5)
).astype('float32')

# --- 8) (권장) 절기 내 단조(비감소) 보정 ---
train_st = train_st.sort_values(['영업장명_메뉴명','solar_term','term_cycle_id','day_of_term'])
train_st['cum_share_ref_solar'] = (
    train_st.groupby(['영업장명_메뉴명','solar_term','term_cycle_id'])['cum_share_ref_solar']
            .apply(lambda s: s.ffill().bfill().cummax().clip(0,1))
            .reset_index(level=[0,1,2], drop=True)
)

# --- 9) 원본 train에 최종 칼럼만 부착 & 보조컬럼 정리 ---
train = train.copy()
train['cum_share_ref_solar'] = train_st.sort_index()['cum_share_ref_solar'].values
# 필요 없으면 보조 컬럼 정리(원본 train에는 없을 수 있으므로 ignore)
train = train.drop(columns=['term_cycle_id','day_of_term'], errors='ignore')

In [714]:
train.columns

Index(['영업일자', '영업장명_메뉴명', '매출수량', 'time_idx', 'year', 'month', 'day', '영업장명',
       '메뉴명', 'weekday', 'is_holiday', 'is_sandwich', 'is_before_holiday',
       'is_after_holiday', 'is_weekend', 'is_popular', 'menu_category',
       'avg_sales_all_days', 'avg_sales_nonzero_days', 'zero_sales_day_ratio',
       'is_sparse_menu', 'is_drink', 'is_alcohol', 'is_set_menu',
       'demand_volatility', 'demand_stability', 'is_nonzero', 'quarter',
       'day_of_quarter', 'cum_share_ref', 'cum_share_ref_season', 'season',
       'cum_share_ref_solar'],
      dtype='object')

# **미라시아 단체 관련 변수( brunch_flag, hallroom_flag)**

**brunch_flag**: 단체 브런치 메뉴 매출이 생긴 날의 플래그, 연회장_룸타입('Grand Ballroom', 'Convention Hall', 'Conference L', 'Conference M', 'OPUS 2')에만 플래그를 세운다.

**hallroom_flag**: 연회장_룸타입 매출이 생긴 날의 플래그, (단체)브런치주중 36,000 에만 플래그를 세운다.

In [715]:
TARGET = '미라시아_(단체)브런치주중 36,000'
HALL_ROOMS = {'Grand Ballroom', 'Convention Hall', 'Conference L', 'Conference M', 'OPUS'}

# 날짜형 변환
if train['영업일자'].dtype == 'O':
    train['영업일자'] = pd.to_datetime(train['영업일자'], errors='coerce')

# '영업장명_메뉴명' 분리
tokens = train['영업장명_메뉴명'].str.split('_', n=1, expand=True)
store0 = tokens[0].astype(str).str.strip()   # 예: '연회장', '미라시아', ...
store1 = tokens[1].astype(str).str.strip()   # 예: 'Grand Ballroom', '(단체)브런치주중 36,000', ...

# 1) 연회장 매출 발생 날짜
hall_mask = (store0.eq('연회장')) & (store1.isin(HALL_ROOMS)) & (train['매출수량'] > 0)
hall_dates = train.loc[hall_mask, '영업일자'].unique()

# 2) 브런치 매출 발생 날짜
brunch_mask = train['영업장명_메뉴명'].eq(TARGET) & (train['매출수량'] > 0)
brunch_dates = train.loc[brunch_mask, '영업일자'].unique()

# 3) 플래그 생성 (반대로 반영)
# 초기화: 전부 0
train['brunch_flag'] = 0      # ← 연회장 라인에 찍힘 (브런치 매출 발생일 기준)
train['hallroom_flag'] = 0    # ← 미라시아 단체 브런치 라인에 찍힘 (연회장 매출 발생일 기준)

# A) 단체 브런치 매출 발생일 → "연회장_*" 행에 brunch_flag=1
train.loc[hall_mask & train['영업일자'].isin(brunch_dates), 'brunch_flag'] = 1

# B) 연회장 매출 발생일 → "미라시아_(단체)브런치주중 36,000" 행에 hallroom_flag=1
target_row_mask = train['영업장명_메뉴명'].eq(TARGET)
train.loc[target_row_mask & train['영업일자'].isin(hall_dates), 'hallroom_flag'] = 1

In [716]:
# Add features from the 'train' DataFrame to 'df'
# Ensure '영업일자' is datetime in both dataframes for merging
train['영업일자'] = pd.to_datetime(train['영업일자'], errors='coerce')

# Merge based on '영업일자' and '영업장명_메뉴명'
# Recreate '영업장명_메뉴명' in train for merging if it was dropped
if '영업장명_메뉴명' not in train.columns:
    train['영업장명_메뉴명'] = train['영업장명'] + '_' + train['메뉴명']

# Add banquet_type from df_연회장
# Ensure '영업일자' is datetime in df_연회장
df_연회장['영업일자'] = pd.to_datetime(df_연회장['영업일자'], errors='coerce')

train = pd.merge(train, df_연회장[['영업일자', 'banquet_type']], on='영업일자', how='left')

# Fill NaN values in banquet_type with 0 (assuming 0 means no banquet)
train['banquet_type'] = train['banquet_type'].fillna(0).astype(int)

# Store trian.csv

In [717]:
# train.drop(columns=['영업장명','메뉴명'], errors='ignore', inplace=True)
train[['영업장명', '메뉴명']] = train['영업장명_메뉴명'].str.split('_', expand=True)
# train.drop(columns=['day_of_quarter'], inplace=True, errors='ignore')
# train.drop(columns=['cum_share_ref'], inplace=True, errors='ignore')
# train DataFrame을 CSV로 저장 (모든 feature 포함)
train.to_csv("re_train_05.csv", index=False, encoding="utf-8-sig")

print(train.columns)

print("train DataFrame이 output.csv로 저장되었습니다.")

Index(['영업일자', '영업장명_메뉴명', '매출수량', 'time_idx', 'year', 'month', 'day', '영업장명',
       '메뉴명', 'weekday', 'is_holiday', 'is_sandwich', 'is_before_holiday',
       'is_after_holiday', 'is_weekend', 'is_popular', 'menu_category',
       'avg_sales_all_days', 'avg_sales_nonzero_days', 'zero_sales_day_ratio',
       'is_sparse_menu', 'is_drink', 'is_alcohol', 'is_set_menu',
       'demand_volatility', 'demand_stability', 'is_nonzero', 'quarter',
       'day_of_quarter', 'cum_share_ref', 'cum_share_ref_season', 'season',
       'cum_share_ref_solar', 'brunch_flag', 'hallroom_flag', 'banquet_type'],
      dtype='object')
train DataFrame이 output.csv로 저장되었습니다.


# Task
Apply the same feature engineering steps (date features, holiday features, spike/drop, seasonal index, banquet type, etc.) that were applied to the training data to the following test files: "TEST_01.csv", "TEST_02.csv", "TEST_03.csv", "TEST_04.csv", "TEST_05.csv", "TEST_06.csv", "TEST_07.csv", "TEST_08.csv", "TEST_09.csv". Ensure that all engineered features are added as new columns to the respective DataFrames loaded from these files.

## Identify test files

### Subtask:
Create a list of all the test file paths (`TEST_01.csv` to `TEST_09.csv`).


**Reasoning**:
Create a list containing the file paths for the test datasets.



In [718]:
test_files = [f"TEST_{i:02d}.csv" for i in range(0, 10)] # Changed range from 1 to 0 to include TEST_00
print(test_files)

['TEST_00.csv', 'TEST_01.csv', 'TEST_02.csv', 'TEST_03.csv', 'TEST_04.csv', 'TEST_05.csv', 'TEST_06.csv', 'TEST_07.csv', 'TEST_08.csv', 'TEST_09.csv']


## Define feature engineering function

### Subtask:
Create a function that takes a DataFrame (like the one loaded from a test file) and applies all the necessary feature engineering steps (date features, holiday features, spike/drop, seasonal index, banquet type, etc.) to it, returning the processed DataFrame.


**Reasoning**:
Define a function `engineer_features` that takes a DataFrame and applies all the feature engineering steps. This function will include date features, holiday features, spike/drop detection, weekday/weekend price flags, seasonal index, brunch/hallroom flags, and banquet type merging.



In [722]:
def engineer_features(df_test, holiday_df, banquet_df, terms_df=None, terms_csv_path=None):
    """Applies feature engineering steps to a test DataFrame.
       terms_df: (선택) 절기 테이블 DataFrame (cols: ['locdate','solar_term'])
       terms_csv_path: (선택) terms_df가 없을 때 읽어올 CSV 경로
    """
    import pandas as pd
    df_test[['영업장명', '메뉴명']] = df_test['영업장명_메뉴명'].str.split('_', expand=True)


    # --- 기본 날짜 피처 ---
    df_test = df_test.copy()
    df_test['영업일자'] = pd.to_datetime(df_test['영업일자'], errors='coerce')
    df_test['time_idx'] = (df_test['영업일자'] - df_test['영업일자'].min()).dt.days
    df_test['year']    = df_test['영업일자'].dt.year.astype(int)
    df_test['month']   = df_test['영업일자'].dt.month.astype(int)
    df_test['day']     = df_test['영업일자'].dt.day.astype(int)
    df_test['weekday'] = df_test['영업일자'].dt.weekday.astype(int)

    # --- season 코드 (이미 사용 중) ---
    def get_season_code(m):
        if m in (12,1,2): return 0
        if m in (3,4,5):  return 1
        if m in (6,7,8):  return 2
        return 3
    df_test['season'] = df_test['month'].apply(get_season_code)

    # --- 시즌 피처 생성 (season, season_year, day_of_season) ---
    df_test_s = add_season_feats(df_test, '영업일자')

    # --- 1차 매핑: (매장메뉴, day_of_season) 시즌 참조 ---
    #  * train 쪽에서 만든 curve_ref_season, global_md_dos, global_md_month_day를 사용
    #  * 이름 충돌 방지 위해 그대로 사용(위 train 코드 그대로 실행되어 있다고 가정)
    df_test_s = df_test_s.merge(
        curve_ref_season, on=['영업장명_메뉴명','day_of_season'],
        how='left', validate='many_to_one'
    ).merge(
        global_md_dos, on='day_of_season',
        how='left', validate='many_to_one'
    ).merge(
        global_md_month_day, on=['month','day'],
        how='left', validate='many_to_one'
    )

    # --- 최종 합성: dos → dos_global → (month,day) → 0.5 ---
    df_test_s['cum_share_ref_season'] = (
        df_test_s['cum_share_ref_dos']
                .fillna(df_test_s['cum_share_dos_md'])
                .fillna(df_test_s['cum_share_global_md'])
                .fillna(0.5)
    ).astype('float32')

    # --- (권장) 시즌 내 단조 보정 ---
    df_test_s = df_test_s.sort_values(['영업장명_메뉴명','season_year','season','day_of_season'])
    df_test_s['cum_share_ref_season'] = (
        df_test_s.groupby(['영업장명_메뉴명','season_year','season'])['cum_share_ref_season']
                .apply(lambda s: s.ffill().bfill().cummax().clip(0,1))
                .reset_index(level=[0,1,2], drop=True)
    )

    # --- 원래 df_test에 결과만 붙이기(원본 순서 보존) ---
    df_test['cum_share_ref_season'] = df_test_s.sort_index()['cum_share_ref_season'].values

    # =========================
    # SOLAR TERM 매핑 시작
    # =========================
    # 0) terms 준비
    if terms_df is None:
        if not terms_csv_path:
            raise ValueError("terms_df 또는 terms_csv_path를 제공해야 solar_term을 생성할 수 있습니다.")
        terms = pd.read_csv(terms_csv_path)
    else:
        terms = terms_df.copy()

    # 1) 절기 구간 정의
    terms['locdate'] = pd.to_datetime(terms['locdate'], errors='coerce')
    terms_st = terms.sort_values('locdate').reset_index(drop=True)
    terms_st['end_date'] = terms_st['locdate'].shift(-1) - pd.Timedelta(days=1)
    terms_st.loc[terms_st.index[-1], 'end_date'] = df_test['영업일자'].max()

    # 2) asof 머지 (왼쪽 solar_term 삭제해 충돌 방지)
    left  = df_test.drop(columns=['solar_term'], errors='ignore').sort_values('영업일자')
    right = terms_st[['locdate','end_date','solar_term']].sort_values('locdate')
    merged = pd.merge_asof(left, right, left_on='영업일자', right_on='locdate', direction='backward')

    # 3) 구간 내만 유효
    in_range = merged['영업일자'] <= merged['end_date']
    merged['solar_term'] = merged['solar_term'].where(in_range)

    # 4) 사이클/경과일
    df_test_st = merged.rename(columns={'locdate':'term_start','end_date':'term_end'}).copy()
    df_test_st['term_start']  = pd.to_datetime(df_test_st['term_start'])
    df_test_st['term_end']    = pd.to_datetime(df_test_st['term_end'])
    df_test_st['day_of_term'] = (df_test_st['영업일자'] - df_test_st['term_start']).dt.days.astype('Int16')
    df_test_st['term_cycle_id'] = df_test_st['term_start']

    # 5) 백업 키
    df_test_st['month'] = df_test_st['영업일자'].dt.month.astype('Int8')
    df_test_st['day']   = df_test_st['영업일자'].dt.day.astype('Int8')

    # 6) 참조 매핑 (train에서 만든 테이블 사용 가정)
    # curve_ref_term: ['영업장명_메뉴명','solar_term','day_of_term','cum_share_ref_dot']
    # global_md_dot:  ['solar_term','day_of_term','cum_share_term_dot_md']
    # global_md_month_day_term: ['month','day','cum_share_global_md_term']
    df_test_st = df_test_st.merge(
        curve_ref_term, on=['영업장명_메뉴명','solar_term','day_of_term'],
        how='left', validate='many_to_one'
    ).merge(
        global_md_dot, on=['solar_term','day_of_term'],
        how='left', validate='many_to_one'
    ).merge(
        global_md_month_day_term, on=['month','day'],
        how='left', validate='many_to_one'
    )

    df_test_st['cum_share_ref_solar'] = (
        df_test_st['cum_share_ref_dot']
            .fillna(df_test_st['cum_share_term_dot_md'])
            .fillna(df_test_st['cum_share_global_md_term'])
            .fillna(0.5)
    ).astype('float32')

    # 7) 절기 내 단조 보정
    df_test_st = df_test_st.sort_values(['영업장명_메뉴명','solar_term','term_cycle_id','day_of_term'])
    df_test_st['cum_share_ref_solar'] = (
        df_test_st.groupby(['영업장명_메뉴명','solar_term','term_cycle_id'])['cum_share_ref_solar']
                  .apply(lambda s: s.ffill().bfill().cummax().clip(0,1))
                  .reset_index(level=[0,1,2], drop=True)
    )

    # 8) 최종 부착(+ solar_term 컬럼도 함께 복원) & 정리
    df_test['cum_share_ref_solar'] = df_test_st.sort_index()['cum_share_ref_solar'].values
    # ★ 여기서 solar_term을 df_test로 되돌려 붙여줌 (없으면 KeyError 방지)
    df_test['solar_term'] = df_test_st.sort_index()['solar_term'].values
    # 원하면 nullable 정수로 캐스팅
    df_test['solar_term'] = df_test['solar_term'].astype('Int64')

    # 보조 컬럼 정리
    df_test = df_test.drop(columns=['term_cycle_id','day_of_term'], errors='ignore')

    # 2. Holiday features
    holiday_df['locdate'] = pd.to_datetime(holiday_df['locdate'])
    df_test = pd.merge(
        df_test,
        holiday_df[['locdate', 'isHoliday']],
        how='left',
        left_on='영업일자',
        right_on='locdate'
    )
    df_test['is_holiday'] = df_test['isHoliday'].fillna('N').apply(lambda x: 1 if x == 'Y' else 0)
    df_test = df_test.drop(['locdate', 'isHoliday'], axis=1)  # 둘 다 삭제

    df_test['is_before_holiday'] = df_test['is_holiday'].shift(-1).fillna(0).astype(int)
    df_test['is_after_holiday'] = df_test['is_holiday'].shift(1).fillna(0).astype(int)
    df_test['is_weekend'] = df_test['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)
    df_test["is_sandwich"] = 0
    df_test.loc[
        (df_test["is_holiday"] == 0) &
        (df_test["is_holiday"].shift(1) == 1) &
        (df_test["is_holiday"].shift(-1) == 1),
        "is_sandwich"
    ] = 1

    # 3. lookup features

    feat_cols = ['영업장명_메뉴명',
             'menu_category','avg_sales_all_days','avg_sales_nonzero_days',
             'zero_sales_day_ratio','is_sparse_menu','is_drink','is_alcohol',
             'is_set_menu','demand_volatility','demand_stability', 'is_popular']

    train_feats = (train[feat_cols]
               .groupby('영업장명_메뉴명', as_index=False)
               .agg('first'))  # 필요시 mean/max로 변경

    df_test = df_test.merge(train_feats, on='영업장명_메뉴명',
                        how='left', validate='many_to_one')
    

    # --- 선행: quarter, month/day, day_of_quarter 생성 (있으면 건너뜀) ---
    if 'quarter' not in df_test.columns:
        df_test['quarter'] = df_test['영업일자'].dt.to_period('Q').astype(str).str[-2:]
    df_test['quarter'] = df_test['quarter'].map({'Q1':0,'Q2':1,'Q3':2,'Q4':3}).astype('Int8')

    if 'year' not in df_test.columns:
        df_test['year'] = df_test['영업일자'].dt.year.astype('Int16')
    if 'month' not in df_test.columns:
        df_test['month'] = df_test['영업일자'].dt.month.astype('Int8')
    if 'day' not in df_test.columns:
        df_test['day'] = df_test['영업일자'].dt.day.astype('Int8')

    q = df_test['영업일자'].dt.to_period('Q')
    q_start = q.dt.start_time
    df_test['day_of_quarter'] = (df_test['영업일자'] - q_start).dt.days.astype('Int16')

    # --- df_train_subset: (영업장명_메뉴명, month, day)별 첫 값만 ---
    train_merge_cols = ['영업장명_메뉴명','month','day','cum_share_ref']
    df_train_subset = (
        train[train_merge_cols]
        .groupby(['영업장명_메뉴명','month','day'], as_index=False)
        .first()
        .rename(columns={'cum_share_ref':'cum_share_ref_md'})  # ← 이름 분리
    )

    # 타입 맞추기
    df_train_subset['month'] = df_train_subset['month'].astype('Int8')
    df_train_subset['day']   = df_train_subset['day'].astype('Int8')

    # ---- 수정 ----
    cr_q = curve_ref.copy()
    cr_q = cr_q.rename(columns={'cum_share_ref': 'cum_share_ref_doq'})
    cr_q['day_of_quarter'] = cr_q['day_of_quarter'].astype('Int16')


    # --- 1차 매핑: (매장메뉴, day_of_quarter) ---
    df_test = df_test.merge(
        cr_q,
        on=['영업장명_메뉴명','day_of_quarter'],
        how='left',
        validate='many_to_one'  # 1:N 폭증 방지
    )

    # --- 2차 매핑(백오프): (매장메뉴, month, day) 첫 값 ---
    df_test = df_test.merge(
        df_train_subset,
        on=['영업장명_메뉴명','month','day'],
        how='left',
        validate='many_to_one'
    )

    # --- 최종 합성: doq 우선 → md 백오프 → 기본값 0.5 ---
    df_test['cum_share_ref'] = (
        df_test['cum_share_ref_doq']
            .fillna(df_test['cum_share_ref_md'])
            .fillna(0.5)
    ).astype('float32')

    # --- (강력 추천) 분기 내 단조 증가 보정 ---
    df_test = df_test.sort_values(['영업장명_메뉴명','year','quarter','영업일자'])
    df_test['cum_share_ref'] = (
        df_test.groupby(['영업장명_메뉴명','year','quarter'])['cum_share_ref']
            .apply(lambda s: s.ffill().bfill().cummax().clip(0, 1))
            .reset_index(level=[0,1,2], drop=True)
    )

    # 임시 컬럼 정리(원하면 남겨도 됨)
    df_test = df_test.drop(columns=['cum_share_ref_doq','cum_share_ref_md'], errors='ignore')

    # 6. Brunch/Hallroom flags
    TARGET = '미라시아_(단체)브런치주중 36,000'
    HALL_ROOMS = {'Grand Ballroom', 'Convention Hall', 'Conference L', 'Conference M', 'OPUS'}

    tokens = df_test['영업장명_메뉴명'].str.split('_', n=1, expand=True)
    store0 = tokens[0].astype(str).str.strip()
    store1 = tokens[1].astype(str).str.strip()

    # Recalculate hall_dates and brunch_dates from the *original train* data (assuming 'train' df is available globally)
    if 'train' in globals():
        train_tokens = train['영업장명_메뉴명'].str.split('_', n=1, expand=True)
        train_store0 = train_tokens[0].astype(str).str.strip()
        train_store1 = train_tokens[1].astype(str).str.strip()

        train_hall_mask = (train_store0.eq('연회장')) & (train_store1.isin(HALL_ROOMS)) & (train['매출수량'] > 0)
        hall_dates = train.loc[train_hall_mask, '영업일자'].unique()

        train_brunch_mask = train['영업장명_메뉴명'].eq(TARGET) & (train['매출수량'] > 0)
        brunch_dates = train.loc[train_brunch_mask, '영업일자'].unique()
    else:
        # Fallback or error handling if train data is not available
        print("Warning: 'train' DataFrame not found. Cannot calculate brunch_dates and hall_dates.")
        hall_dates = []
        brunch_dates = []


    if 'brunch_flag' not in df_test.columns:
        df_test['brunch_flag'] = 0
    if 'hallroom_flag' not in df_test.columns:
        df_test['hallroom_flag'] = 0

    test_hall_mask = (store0.eq('연회장')) & (store1.isin(HALL_ROOMS))
    test_target_row_mask = df_test['영업장명_메뉴명'].eq(TARGET)

    df_test.loc[test_hall_mask & df_test['영업일자'].isin(brunch_dates), 'brunch_flag'] = 1
    df_test.loc[test_target_row_mask & df_test['영업일자'].isin(hall_dates), 'hallroom_flag'] = 1


    # 7. Banquet type
    # Ensure '영업일자' is datetime in banquet_df
    banquet_df['영업일자'] = pd.to_datetime(banquet_df['영업일자'], errors='coerce')
    df_test = pd.merge(df_test, banquet_df[['영업일자', 'banquet_type']], on='영업일자', how='left')
    df_test['banquet_type'] = df_test['banquet_type'].fillna(0).astype(int)


    return df_test

## Process each test file

### Subtask:
Iterate through the list of test file paths. For each file:
    - Load the CSV into a DataFrame.
    - Apply the feature engineering function to the DataFrame.
    - Store the processed DataFrame (e.g., in a dictionary or list).


**Reasoning**:
Iterate through the test files, apply the feature engineering function to each, and store the results.



In [723]:
processed_test_dfs = {}

# Load necessary dataframes outside the loop
print(os.getcwd())  # Ensure the current working directory is set correctly
terms_df = pd.read_csv('../solar_term_2023_2025.csv')  # Load the terms CSV
holiday_df = pd.read_csv('../holidays_2023_2025.csv')
banquet_df_full = pd.read_csv('re_train_05.csv') # Load the processed train data

# Prepare the banquet_df for merging (only date and type)
banquet_df_for_merge = banquet_df_full[['영업일자', 'banquet_type']].drop_duplicates()

for file_path in test_files:
    print(f"Processing {file_path}...")
    # Load the test data
    df_test = pd.read_csv(f"../../data/test/{file_path}")

    # Apply feature engineering
    # Pass the necessary dataframes to the function
    processed_df = engineer_features(df_test, holiday_df, banquet_df_for_merge, terms_df)

    # Store the processed DataFrame
    processed_test_dfs[file_path] = processed_df
    print(f"Finished processing {file_path}.")

# Display the first few rows of one of the processed dataframes to verify
if processed_test_dfs:
    first_file = list(processed_test_dfs.keys())[0]
    print(f"\nSample of processed {first_file}:")
    print(processed_test_dfs[first_file].columns.tolist())

/Users/garden/Desktop/lgaimers/Hackaton/DataProcessing/re_data_processed
Processing TEST_00.csv...
Finished processing TEST_00.csv.
Processing TEST_01.csv...
Finished processing TEST_01.csv.
Processing TEST_02.csv...
Finished processing TEST_02.csv.
Processing TEST_03.csv...
Finished processing TEST_03.csv.
Processing TEST_04.csv...
Finished processing TEST_04.csv.
Processing TEST_05.csv...
Finished processing TEST_05.csv.
Processing TEST_06.csv...
Finished processing TEST_06.csv.
Processing TEST_07.csv...
Finished processing TEST_07.csv.
Processing TEST_08.csv...
Finished processing TEST_08.csv.
Processing TEST_09.csv...
Finished processing TEST_09.csv.

Sample of processed TEST_00.csv:
['영업일자', '영업장명_메뉴명', '매출수량', '영업장명', '메뉴명', 'time_idx', 'year', 'month', 'day', 'weekday', 'season', 'cum_share_ref_season', 'cum_share_ref_solar', 'solar_term', 'is_holiday', 'is_before_holiday', 'is_after_holiday', 'is_weekend', 'is_sandwich', 'menu_category', 'avg_sales_all_days', 'avg_sales_nonzero

## Save processed test data (optional)

### Subtask:
Save each processed test DataFrame to a new CSV file.


**Reasoning**:
Iterate through the processed test dataframes and save each one to a CSV file with an added suffix.



In [724]:
import os

output_dir = "./re_test_processed_03/" # Save in the current directory

for filename, df_processed in processed_test_dfs.items():
    # Construct output filename by adding '_processed' before the extension
    base, ext = os.path.splitext(filename)
    output_filename = f"{base}_processed{ext}"
    output_path = os.path.join(output_dir, output_filename)

    # Save the DataFrame to CSV
    df_processed.to_csv(output_path, index=False, encoding="utf-8-sig")

    print(f"Saved processed {filename} to {output_path}")

Saved processed TEST_00.csv to ./re_test_processed_03/TEST_00_processed.csv
Saved processed TEST_01.csv to ./re_test_processed_03/TEST_01_processed.csv
Saved processed TEST_02.csv to ./re_test_processed_03/TEST_02_processed.csv
Saved processed TEST_03.csv to ./re_test_processed_03/TEST_03_processed.csv
Saved processed TEST_04.csv to ./re_test_processed_03/TEST_04_processed.csv
Saved processed TEST_05.csv to ./re_test_processed_03/TEST_05_processed.csv
Saved processed TEST_06.csv to ./re_test_processed_03/TEST_06_processed.csv
Saved processed TEST_07.csv to ./re_test_processed_03/TEST_07_processed.csv
Saved processed TEST_08.csv to ./re_test_processed_03/TEST_08_processed.csv
Saved processed TEST_09.csv to ./re_test_processed_03/TEST_09_processed.csv
